In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.ml import feature, evaluation, Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import Row

In [2]:
conf = (SparkConf()
            .setAppName('naive_bayes')
            .setMaster('spark://spark-master:7077')
       )
conf.set("spark.executor.memory", "6g")
conf.set("spark.driver.maxResultSize", "0")
conf.set("spark.sql.shuffle.partitions", "6")
conf.set("spark.default.parallelism", "6")
conf.set("spark.driver.memory", "3g") 

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [4]:
# load original dataset without bootstrapped samples
df = (spark.read.format('csv')
      .option('inferSchema', 'true')
      .option('header', 'true')
      .option('escape', '"')
      .load('hdfs://namenode:9000/data/resampled_df.csv') 
     )

# Data cleaning

In [5]:
import re
@F.udf(returnType=T.StringType())
def clean_input2(s):
    common_strings = ['windows', 'system32', 'cmd.exe', 'sandcat.exe', 'c', 'windowspowershell', 'powershell.exe', '']
    
    pattern = re.compile(r"""
        [:|"?']
        | --field-trial-handle=.*\d+
        | //.*com(/.*)/
        | \s*"\s*
        | \{.*\}
        | [=;(),]
        | \\
        | //
        | \s+\.\s+ 
        | $.
        | $_.
    """, re.VERBOSE)
    
    return ','.join([x.lower() for x in re.sub(pattern, ' ', s).split() if x.lower() not in common_strings and len(x) >= 2])

In [8]:
from pyspark.sql.functions import regexp_extract, regexp_replace, col, count, split, size, to_date

def clean_input1(df):
    _df = df.select(
        '*'
    ).withColumn(
        'cmd_line_tokens', split(clean_input2(col('command_line')), ',')
    )
    return _df

In [9]:
df = clean_input1(df)

In [10]:
df.select('cmd_line_tokens').show(5, False)

+-----------------------------------------------------------------+
|cmd_line_tokens                                                  |
+-----------------------------------------------------------------+
|[svchost.exe, -k, localservicenonetwork, -p]                     |
|[svchost.exe, -k, localservice, -p, -s, dispbrokerdesktopsvc]    |
|[oobe, windeploy.exe]                                            |
|[oobe, setup.exe]                                                |
|[svchost.exe, -k, localservicenetworkrestricted, -p, -s, lmhosts]|
+-----------------------------------------------------------------+
only showing top 5 rows



# Feature engineering
+ Term Frequency (one-hot): value indicates if feature is present in observation

In [11]:
training_df, validation_df, testing_df = df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [12]:
cv_transformer = feature.CountVectorizer(minTF=1, minDF=5, binary=True, inputCol='cmd_line_tokens', outputCol='tf')

In [13]:
cv_estimator = cv_transformer.fit(training_df)

In [14]:
len(cv_estimator.vocabulary)

1827

# Modeling

In [15]:
nb = NaiveBayes(featuresCol='tf', labelCol='class_label', smoothing=1, modelType='bernoulli')
nb_estimator = Pipeline(stages=[cv_transformer, nb])
nb_model = nb_estimator.fit(training_df)

In [16]:
nb_model.transform(testing_df).\
    select(F.avg(F.expr('float(class_label = prediction)')).alias('accuracy')).\
    first()

Row(accuracy=0.9806562689892647)

# Model Tuning

In [17]:
paramGrid = (ParamGridBuilder() 
                 .addGrid(nb_model.stages[0].minDF, [1, 5, 10, 20, 50, 75, 100]) 
                 .build()
            )

In [18]:
models = []
for grid in range(len(paramGrid)):
    print("Fitting model {}".format(grid))
    _model = nb_estimator.fit(validation_df, paramGrid[grid])
    models.append(_model)

Fitting model 0
Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6


In [19]:
evaluator = BinaryClassificationEvaluator(labelCol='class_label', metricName='areaUnderROC')
auc_scores = [evaluator.evaluate(model.transform(validation_df)) for model in models]

In [20]:
auc_scores

[0.8622502140595931,
 0.8558209498105585,
 0.8472067544661606,
 0.8479586210203298,
 0.8676173840647936,
 0.8648744956001704,
 0.8606494671346981]

In [21]:
best_model_idx = np.argmax(auc_scores)
best_model = models[best_model_idx]
print("Best params: \n\n{}\n".format(paramGrid[best_model_idx]))
print("Best AUC: \n\n{}".format(auc_scores[best_model_idx]))

Best params: 

{Param(parent='CountVectorizer_d8f611e5646a', name='minDF', doc='Specifies the minimum number of different documents a term must appear in to be included in the vocabulary. If this is an integer >= 1, this specifies the number of documents the term must appear in; if this is a double in [0,1), then this specifies the fraction of documents. Default 1.0'): 50.0}

Best AUC: 

0.8676173840647936


## Best model
+ minDF: 25

# Cross Validation
+ test best_model performance on training dataset

In [22]:
evaluator.evaluate(best_model.transform(training_df))

0.8671537345481183